In [ ]:
import random
import numpy as np
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

from model import CharacterCNN
from model import weights_init
from model import DatasetReader

# Set random seem for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
th.manual_seed(manualSeed)

In [ ]:
#Character vocab
char_inventory = 'abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:\'"/\\|_@#$%ˆ&*˜`+-=<>()[]{}\n'

vocab = {c:i for i,c in enumerate(char_inventory)}

# One hot embedding size
nchars = len(char_inventory)

#Input length
input_length = 1014

# Decide which device we want to run on
ngpu = 1
device = th.device("cuda:0" if (th.cuda.is_available() and ngpu > 0) else "cpu")

#Batch size
batch_size = 128

#Number of threads for the data loader
workers = 2

#Number of epochs
nepochs = 20

In [ ]:
cnn = CharacterCNN(nchars).to(device)
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
cnn.apply(weights_init)
cnn.train()
# Print the model
print(cnn)

In [ ]:
train_data = DatasetReader('./data/yelp_review_full_csv/train.csv', vocab, input_length, nchars)
dataloader = th.utils.data.DataLoader(train_data, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)



In [ ]:
# The objective
criterion = nn.CrossEntropyLoss()

#Optimizer
optimizer = optim.Adam(cnn.parameters())

In [ ]:
losses = []
lr = 0.01

#Main training loop
for epoch in range(nepochs):
        
    for i, batch in enumerate(dataloader):
        inputs = batch[0].type(th.FloatTensor).to(device)
        targets = batch[1].type(th.LongTensor).to(device)
        
        optimizer.zero_grad()
        preds = cnn(inputs)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()
        
        # Output training stats
        if i % 5 == 0:
            print('[%d/%d][%d/%d]\tLoss: %.4f'
                  % (epoch, nepochs, i, len(dataloader), loss.item()))
            losses.append(loss)
        if i % 100 == 0:
            th.save({
                'model': cnn.state_dict(),
                'epoch': epoch,
            }, './output/cnn.pth')
th.save({
    'model': cnn.state_dict(),
    'epoch': epoch,
}, './output/cnn.pth')

# Eval

In [ ]:
test_data = DatasetReader('./data/yelp_review_full_csv/test.csv', vocab, input_length, nchars)
dataloader = th.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=workers)


In [ ]:
cnn.load_state_dict(th.load('./output/cnn.pth')['model'])
cnn.eval()

In [ ]:
preds = []
gt = []
for batch in tqdm(dataloader):
    inputs = batch[0].type(th.FloatTensor).to(device)
    targets = batch[1].type(th.LongTensor).to(device)
        
    out = cnn(inputs)
    preds.extend(list(np.argmax(out.cpu().data.numpy(), axis=1)))
    gt.extend(list(targets.squeeze().cpu().data.numpy()))

In [ ]:
err_rate = sum([preds[i] != gt[i] for i in range(len(gt))] ) / len(gt)
print("Error rate: %.4f"%err_rate)